In [2]:
import matplotlib.pyplot as plt
import cv2
import cv_algorithms

imgBin = cv2.imread("2.3-road_dil_bin.png", cv2.IMREAD_GRAYSCALE)

# Add padding to the image. This is necessary because on borders of the image the thick lines touching the border will not be thinned. The padding should be done.
padded = cv2.copyMakeBorder(
    imgBin, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=[0, 0, 0])

# Perform thinning out-of-place
guoHallThinning = cv_algorithms.zhang_suen(padded)
# ... or allow the library to modify the original image (= faster):
# cv_algorithms.guo_hall(imgThresh, inplace=True)

# remove padding
# guoHallThinning = guoHallThinning[10:-10, 10:-10]

cv2.imwrite("3-skeletonized_roadmap.png", guoHallThinning)


True

In [3]:
import cv2
import networkx as nx
import numpy as np
import numpy
from collections import defaultdict
from itertools import chain


def zhang_suen_node_detection(skel):
    """
    (from nefi1)
    Node detection based on criteria put forward in "A fast parallel algorithm
    for thinning digital patterns" by T. Y. Zhang and C. Y. Suen. Pixels p of
    the skeleton are categorized as nodes/non-nodes based on the value of a
    function A(p) depending on the pixel neighborhood of p. Please check the
    above paper for details.

    A(p1) == 1: The pixel p1 sits at the end of a skeleton line, thus a node
    of degree 1 has been found.
    A(p1) == 2: The pixel p1 sits in the middel of a skeleton line but not at
    a branching point, thus a node of degree 2 has been found. Such nodes are
    ignored and not introduced to the graph.
    A(p1) >= 3: The pixel p1 belongs to a branching point of a skeleton line,
    thus a node of degree >=3 has been found.

    Args:
        *skel* : Skeletonised source image. The skeleton must be exactly 1
         pixel wide.

    Returns:
        *graph* : networkx Graph object with detected nodes.

    """
    def check_pixel_neighborhood(x, y, skel):
        """
        Check the number of components around a pixel.
        If it is either 1 or more than 3, it is a node.

        Args:
            | *x* : pixel location value
            | *y* : pixel location value
            | *skel* : skeleton Graph object

        Returns:
            *accept_pixel_as_node* : boolean value

        """
        accept_pixel_as_node = False
        item = skel.item
        p2 = item(x - 1, y) / 255
        p3 = item(x - 1, y + 1) / 255
        p4 = item(x, y + 1) / 255
        p5 = item(x + 1, y + 1) / 255
        p6 = item(x + 1, y) / 255
        p7 = item(x + 1, y - 1) / 255
        p8 = item(x, y - 1) / 255
        p9 = item(x - 1, y - 1) / 255

        # The function A(p1),
        # where p1 is the pixel whose neighborhood is beeing checked
        components = (p2 == 0 and p3 == 1) + (p3 == 0 and p4 == 1) + \
                     (p4 == 0 and p5 == 1) + (p5 == 0 and p6 == 1) + \
                     (p6 == 0 and p7 == 1) + (p7 == 0 and p8 == 1) + \
                     (p8 == 0 and p9 == 1) + (p9 == 0 and p2 == 1)
        if (components >= 3) or (components == 1):
            accept_pixel_as_node = True
        return accept_pixel_as_node

    graph = nx.Graph()
    w, h = skel.shape
    item = skel.item
    for x in range(1, w - 1):
        for y in range(1, h - 1):
            if item(x, y) != 0 and check_pixel_neighborhood(x, y, skel):
                graph.add_node((x, y))
    return graph


def breadth_first_edge_detection(skel, segmented, graph):
    """
    (from nefi1)
    Detect edges in the skeletonized image.
    Also compute the following edge properties:

        | *pixels* : number of pixels on the edge in the skeleton
        | *length* : length in pixels, horizontal/vertikal steps count 1,
           diagonal steps count sqrt 2
        | *width* : the mean diameter of the edge
        | *width_var* : the variance of the width along the edge

    The runtime is linear in the number of pixels.
    White pixels are **much more** expensive though.
    """
    def neighbors(x, y, label):
        item = skel.item
        print(label, item(x, y))
        width, height = skel.shape
        for dy in [-1, 0, 1]:
            for dx in [-1, 0, 1]:
                if (dx != 0 or dy != 0) and \
                        0 <= x + dx < width and \
                        0 <= y + dy < height and \
                        item(x + dx, y + dy) != 0:
                    yield x + dx, y + dy

    def distance_transform_diameter(edge_trace, segmented):
        """
        (dev comments from nefi1)
        my cv2 lacks cv2.DIST_L2, it seems to have the value 2 though, so I use
        that, same for MASK_PRECISE
        <python3 cv2.DIST_L2 equals to 2>
        """
        dt = cv2.distanceTransform(segmented, 2, 0)
        edge_pixels = np.nonzero(edge_trace)
        diameters = defaultdict(list)
        for label, diam in zip(edge_trace[edge_pixels], 2.0 * dt[edge_pixels]):
            diameters[label].append(diam)
        return diameters

    # compute edge length
    # initialize: the neighbor pixels of each node get a distinct label
    # each label gets a queue
    label_node = dict()
    queues = []
    label = 1
    label_length = defaultdict(int)
    print(graph.nodes())
    for x, y in graph.nodes():
        for a, b in neighbors(x, y, label):
            label_node[label] = (x, y)
            label_length[label] = 1.414214 if abs(x - a) == 1 and abs(y - b) == 1 \
                else 1
            queues.append((label, (x, y), [(a, b)]))
            label += 1
    print(queues)
    # bfs over the white pixels.
    # One phase: every entry in queues is handled
    # Each label grows in every phase.
    # If two labels meet, we have an edge.
    edges = set()
    edge_trace = np.zeros(skel.shape, np.uint32)
    edge_value = edge_trace.item
    edge_set_value = edge_trace.itemset
    label_histogram = defaultdict(int)

    while queues:
        new_queues = []
        for label, (px, py), nbs in queues:
            for (ix, iy) in nbs:
                value = edge_value(ix, iy)
                if value == 0:
                    edge_set_value((ix, iy), label)
                    label_histogram[label] += 1
                    # TODO consider using cv2.arcLength for this
                    label_length[label] += 1.414214 if abs(ix - px) == 1 and \
                        abs(iy - py) == 1 else 1
                    new_queues.append(
                        (label, (ix, iy), neighbors(ix, iy, label)))
                elif value != label:
                    edges.add((min(label, value), max(label, value)))
        queues = new_queues

    # compute edge diameters
    diameters = distance_transform_diameter(edge_trace, segmented)
    # add edges to graph
    for l1, l2 in edges:
        u, v = label_node[l1], label_node[l2]
        if u == v:
            continue
        d1, d2 = diameters[l1], diameters[l2]
        diam = np.fromiter(chain(d1, d2), np.uint, len(d1) + len(d2))
        graph.add_edge(u, v, pixels=label_histogram[l1] + label_histogram[l2],
                       length=label_length[l1] + label_length[l2],
                       width=np.mean(diam),
                       width_var=np.var(diam))
    return graph


In [4]:
NODESIZESCALING = 750
EDGETRANSPARENCYDIVIDER = 5
EDGETRANSPARENCY = False


def find_max_edge_deviation(graph):
    """
    This methode calculates for each edge its standard deviation and also
    tracks the maximum standard deviation among all edges.
    The maximum standard deviation will then be stored in the graph.
    """
    max_standard_deviation = 0
    for (x1, y1), (x2, y2) in graph.edges():
        deviation = graph[(x1, y1)][(x2, y2)]['width_var']
        standard_deviation = numpy.sqrt(deviation)
        graph[(x1, y1)][(x2, y2)]['standard_deviation'] = standard_deviation

        if max_standard_deviation < standard_deviation:
            max_standard_deviation = standard_deviation

    return max_standard_deviation


def draw_graph(image, graph):
    """
    Draw the graph on the image by traversing the graph structure.

    Args:
        | *image* : the image where the graph needs to be drawn
        | *graph* : the *.txt file containing the graph information

    Returns:

    """
    tmp = draw_edges(image, graph)
    node_size = int(numpy.ceil((max(image.shape) / float(NODESIZESCALING))))
    return draw_nodes(tmp, graph, max(node_size, 1))


def draw_nodes(img, graph, radius=1):
    """
    Draw all nodes on the input image.

    Args:
        | *img* : Input image where nodes are drawn
        | *graph* : Input graph containing the nodes

    Kwargs:
        | *radius* : Radius of drawn nodes

    Returns:
        Input image img with nodes drawn into it
    """
    for x, y in graph.nodes():
        cv2.rectangle(img, (y - radius, x - radius), (y + radius, x + radius),
                      (255, 0, 0), -1)
    return img


def draw_edges(img, graph, col=(0, 0, 255)):
    """
    Draw network edges on the input image.

    Args:
        | *img* : Input image where edges are drawn
        | *graph* : Input graph containing the edges
    Kwargs:
        | *col* : colour for drawing

    Returns:
        Input image img with nodes drawn into it
    """
    edg_img = numpy.copy(img)

    max_standard_deviation = 0
    if EDGETRANSPARENCY:
        max_standard_deviation = find_max_edge_deviation(graph)

    for (x1, y1), (x2, y2) in graph.edges():
        start = (y1, x1)
        end = (y2, x2)
        diam = graph[(x1, y1)][(x2, y2)]['width']
        # variance value computed during graph detection
        width_var = graph[(x1, y1)][(x2, y2)]['width_var']
        # compute edges standard deviation by applying sqrt(var(edge))
        standard_dev = numpy.sqrt(width_var)
        if diam == -1:
            diam = 2
        diam = int(round(diam))
        if diam > 255:
            print(
                'Warning: edge diameter too large for display. Diameter has been reset.')
            diam = 255
        if EDGETRANSPARENCY:
            edge_cur_standard_deviation = graph[(
                x1, y1)][(x2, y2)]['standard_deviation']

            # calculate the opacity based on the condition opacity_max_standard_deviation = 0.8
            opacity = edge_cur_standard_deviation / max_standard_deviation * 0.8

            # access color triple
            (b, g, r) = col

            # set overlay in this case white
            overlay = (0, 0, 0)
            # compute target color based on the transparency formula
            target_col = (b == 0 if 0 else opacity * 255 + (1 - opacity) * b,
                          g == 0 if 0 else opacity * 255 + (1 - opacity) * g,
                          r == 0 if 0 else opacity * 255 + (1 - opacity) * r)

            # draw the line
            cv2.line(edg_img, start, end, target_col, diam)

        else:
            # simply draw a red line since we are not in the edge transparency mode
            cv2.line(edg_img, start, end, col, diam)

    edg_img = cv2.addWeighted(img, 0.5, edg_img, 0.5, 0)

    MAXIMUMSTANDARDDEVIATION = 0

    return edg_img



In [5]:
skeleton = cv2.imread("3-skeletonized_roadmap.png", cv2.IMREAD_GRAYSCALE)
image = cv2.imread("2.3-road_dil_bin.png", cv2.IMREAD_GRAYSCALE)

# add padding to get nodes from the image borders which start from white
# skeleton = cv2.copyMakeBorder(
#     skeleton, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=[0, 0, 0])
image = cv2.copyMakeBorder(image, 1, 1, 1, 1,
                           cv2.BORDER_CONSTANT, value=[0, 0, 0])

# detect nodes
graph = zhang_suen_node_detection(skeleton)

# detect edges
# graph = breadth_first_edge_detection(skeleton, gray_img, graph)
graph = breadth_first_edge_detection(skeleton, skeleton, graph)
# skeleton = cv2.cvtColor(skeleton, cv2.COLOR_GRAY2BGR)

image = cv2.cvtColor(skeleton, cv2.COLOR_GRAY2BGR)
graph_img = draw_graph(image, graph)
cv2.imwrite("zgraph.png", graph_img)


[(14, 22), (17, 16), (18, 26), (20, 12)]
1 255
5 255
8 255
9 255
[(1, (14, 22), [(15, 21)]), (2, (14, 22), [(13, 22)]), (3, (14, 22), [(15, 22)]), (4, (14, 22), [(14, 23)]), (5, (17, 16), [(16, 16)]), (6, (17, 16), [(18, 16)]), (7, (17, 16), [(17, 17)]), (8, (18, 26), [(17, 25)]), (9, (20, 12), [(20, 13)])]
1 255
2 255
3 255
4 255
5 255
6 255
7 255
8 255
9 255
1 255
1 255
1 255
2 255
4 255
5 255
5 255
6 255
7 255
8 255
8 255
9 255
9 255
1 255
1 255
2 255
5 255
2 255
5 255
2 255
5 255
2 255
5 255
5 255


True